In [1]:
from rnamodif.data_utils.dataloading import get_my_valid_dataset_unlimited, get_kfold_split_func, worker_init_fn
from torch.utils.data import IterableDataset, DataLoader

k = 4
dset = get_my_valid_dataset_unlimited(
    window=4096,
    pos_files='pos_2022',
    neg_files='neg_2022',
    split_method=get_kfold_split_func(total_k=5, current_k=k)
)
valid_loader = DataLoader(dset, batch_size=128, num_workers=1, pin_memory=True, persistent_workers=False, worker_init_fn=worker_init_fn)

valid files indicies
[1, 11, 12, 13, 15, 16, 20, 23, 24, 26, 31, 39, 40, 44, 47, 52, 55, 57, 67, 71, 75, 86, 87, 88, 91, 96, 97, 101, 108, 111, 112, 114, 117, 119, 125, 137, 140, 141, 150, 154, 178, 186, 194, 195, 206, 207, 214, 216, 222, 229, 248, 253, 258, 266, 269]
[1, 4, 15, 20, 25, 29, 30, 31, 34, 35, 37, 41, 51, 61, 67, 79, 80, 84, 95, 97, 104, 109, 111, 120, 122, 126, 133, 135, 146, 151, 153, 160, 161, 170, 194, 199, 202, 205, 206, 214, 216, 218, 232, 234, 242, 244, 249, 257, 259, 260, 267, 271, 275, 277, 284, 288, 289, 291, 296, 298, 299, 304, 317, 318, 330, 336, 338, 342, 343, 345, 356, 366, 367, 374, 376, 382, 384, 401, 416, 426, 429, 435, 436, 441, 442, 461, 462, 464, 465, 470, 474, 476, 477, 482, 483, 490, 492, 497, 499, 500, 502, 510, 517, 519]


In [2]:
from rnamodif.architectures.bonito_pretrained import BonitoPretrained
from rnamodif.architectures.rodan_pretrained import RodanPretrained

checkpoint_path = f'checkpoints_pl/rodan_2022_kfold_{k}of5/epoch=0-step=227000.ckpt'
model = RodanPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000).load_from_checkpoint(checkpoint_path).to('cuda')

In [3]:
import numpy as np
def probs_to_bin(probs):
    res = []
    for pos_prob in probs:
        pos_prob = pos_prob[0]
        res.append([1-pos_prob, pos_prob])
    predz = np.array(res)
    return predz

In [ ]:
from cleanlab.filter import find_label_issues
from torch import nn
from pathlib import Path
import pickle

count = 0
logs = []
results_path = Path('mislabelings')
for x,y,identifier in valid_loader:
    count+=1
    # print('count', count)
    #TODO switch model to eval mode to disable dropout etc?
    probs = nn.Softmax(dim=0)(model(x.to('cuda'))).detach().cpu().numpy()
    probs = probs_to_bin(probs)
    identifier['probs'] = probs
    
    identifier_len = len(identifier['label'])
    for i in range(identifier_len):
        new_identifier = {}
        for key,v in identifier.items():
            new_identifier[key] = v[i]
        new_identifier['file'] = Path(new_identifier['file']).stem
        folder = 'pos'
        if(new_identifier['label'] == 0):
            folder='neg'
        file_path = results_path/folder
        with open(file_path/f"{new_identifier['file']}:{new_identifier['readid']}:{new_identifier['start']}",'wb') as f:
            pickle.dump(new_identifier, f)  


FAR89595_30596c4a_75 starting label 1
FAR89595_30596c4a_141 starting label 1
FAR89595_30596c4a_20 starting label 1
FAR89595_30596c4a_96 starting label 1
FAR89595_30596c4a_31 starting label 1
FAR89595_30596c4a_137 starting label 1
FAR89595_30596c4a_117 starting label 1
FAR89595_30596c4a_186 starting label 1
FAR89595_30596c4a_11 starting label 1
FAR89595_30596c4a_248 starting label 1
FAR89595_30596c4a_67 starting label 1
FAR89595_30596c4a_154 starting label 1
FAR89595_30596c4a_88 starting label 1
FAR89595_30596c4a_266 starting label 1
FAR89595_30596c4a_91 starting label 1
FAR89595_30596c4a_24 starting label 1
FAR89595_30596c4a_97 starting label 1
FAR89595_30596c4a_23 starting label 1
FAR89595_30596c4a_26 starting label 1
FAR89595_30596c4a_86 starting label 1
FAR89595_30596c4a_195 starting label 1
FAR89595_30596c4a_55 starting label 1
FAR89595_30596c4a_39 starting label 1
FAR89595_30596c4a_253 starting label 1
FAR89595_30596c4a_87 starting label 1
FAR89595_30596c4a_108 starting label 1
FA

In [ ]:
from pathlib import Path
import pickle

results_path = Path('mislabelings')
positives = list((results_path/'pos').iterdir())
negatives = list((results_path/'neg').iterdir())
logs = positives+negatives


In [ ]:
labels = []
preds = []
#2hours to run
for pth in logs:
    with open(pth,'rb') as f:
        ident = pickle.load(f)
    labels.append(ident['label'].numpy())
    preds.append(ident['probs'])

     

In [ ]:
import numpy as np
from cleanlab.filter import find_label_issues

labels = np.array(labels).astype(np.int32).flatten()

probs = np.array(preds).reshape(-1,2)

print('finding lab issues')  
#TODO run this only once for all predictions
ordered_label_issues = find_label_issues(
    labels=labels,
    pred_probs=probs, 
    # return_indices_ranked_by='self_confidence',
)
   

In [ ]:
#7+ hours
for i, log in enumerate(logs):
    is_mislabeled = ordered_label_issues[i]
    with open(log,'rb') as f:
        ident = pickle.load(f)
    
    ident['is_mislabeled'] = is_mislabeled
    with open(log,'wb') as f:
        pickle.dump(ident, f)  
print('YUP')

In [ ]:
with open(logs[-1],'rb') as f:
    print(logs[-1])
    ident = pickle.load(f)
    print(ident)

In [ ]:
results = {}
for log in logs:
    with open(log,'rb') as f:
        ident = pickle.load(f)
    read_id = str(log).split(':')[1]+f"[{ident['label']}]"
    if(not read_id in results.keys()):
        results[read_id] = [ident['is_mislabeled']]
    else:
        results[read_id] = results[read_id]+[ident['is_mislabeled']]


In [ ]:
import pickle
with open(f'mislabeled_results_2022_{k}of5','wb') as f:
    pickle.dump(results, f)

In [ ]:
import pickle

with open(f'mislabeled_results_2022_{k}of5','rb') as f:
    results = pickle.load(f)

In [7]:
# results

In [ ]:
for k,v in results.items():
    results[k] = sum(v)/len(v)

In [ ]:
pos_mis = [v for k,v in results.items() if int(k[-2]) == 1]
neg_mis = [v for k,v in results.items() if int(k[-2]) == 0]
thresh = 0.96
mis_p = sum([p >= thresh for p in pos_mis])/len(pos_mis)
mis_n = sum([p >= thresh for p in neg_mis])/len(neg_mis)
print(mis_p)
print(format(mis_n, 'f'))


In [ ]:
# read_ids = np.unique([log['readid'] for log in logs])
# results = []
# for read_id in read_ids:
#     filtered_logs = [log for log in logs if log['readid'] == read_id]
#     starts = np.array([log['start'].numpy() for log in filtered_logs])
#     assert len(np.unique(starts)) == len(starts)
#     label = filtered_logs[0]['label']
#     is_mislabeled_map = [log['is_mislabeled'] for log in filtered_logs]
#     is_mislabeled_consensus = sum(is_mislabeled_map)/len(is_mislabeled_map)
#     results.append({
#         'read_id':read_id,
#         'label':label,
#         'is_mislabeled_consensus':is_mislabeled_consensus,
#         'split_into':len(is_mislabeled_map),
#     })
    
# # print('reads:', len(results))
# # [l['label'] for l in results]

In [ ]:
# import pickle

# with open('mislabeled_results_2022_0of5','wb') as f:
#     pickle.dump(results, f)

In [ ]:
# with open('mislabeled_results','rb') as f:
#     results = pickle.load(f)
# results